🌐 Cloudflare Tunnel Demo

**Author:** Katrin Mrotzeck  
**Version:** v1.0.0  
**Last Updated:** 06 February 2026  
**Purpose:** Demo for a working Cloudflare Tunnel returning HTTP request headers.

---


#**Securing a Website through a Cloudflare Tunnel**

>📘 **How to Use This Notebook**
>
>This notebook is designed as a **guided walkthrough and reference implementation**.
>
>- Some cells are **executable only if you own a domain and Cloudflare account**
>- Commands that require real credentials or domains are marked clearly
>- Placeholder values (e.g. `<TUNNEL_NAME>`, `<DOMAIN_NAME>`) must be replaced
>
>You can:
>- Read the notebook end-to-end to understand the full flow
>- Execute selected steps locally if you meet the prerequisites

>⚠️ **Prerequisites**   
>
>This notebook demonstrates a real-world Cloudflare Tunnel setup.
>
>To fully execute all steps, you will need:
>- A registered domain name
>- A Cloudflare account with that domain added
>- Permission to create tunnels and DNS records
>
>If you do not have these, the notebook can still be read
as a step-by-step conceptual walkthrough.

In [ ]:
# Notebook Metadata
AUTHOR = "Katrin Mrotzeck"
VERSION = "v1.0.0"
DATE = "2026-02-06"

print(f"Author: {AUTHOR}")
print(f"Version: {VERSION}")
print(f"Date: {DATE}")

---
# Step by Step Guide


>🧰 **Step 1: Preparing the Environment**
>
>Install all necessary Python packages to ensure the notebook can run all steps.



In [ ]:
# Install required Python packages
!pip install httpbin  # Install the web server (HTTPBin) - note: development server, not for production purposes.
!pip install gunicorn  # Install the web server for real Python apps.


>🧩 **Step 2: Creating the Website**
>
>Create your website domain and connect it with your Cloudflare account.
>
>This step is done outside of this Notebook.

>🚀 **Step 3: Starting the Website**
>
>Start the website locally and let it run in the background.

In [ ]:
# Run the web server using gunicorn in the background
!nohup gunicorn httpbin:app -b 127.0.0.1:5000 &

>🔐 **Step 4: Connecting to Cloudflare**
>
>Load the latest version of cloudflared from the github repository and execute it. Login to your Cloudflare account.

In [ ]:
# Install cloudflared

!curl -L -o cloudflared https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64  # load the latest cloudflared version
!chmod +x cloudflared  # execute cloudflared


>**Login to Cloudflare**
>
>💡 **Tip:** Delete previous tunnels and certificates from the last session to be able to generate a proper new connection. This step is done in the Cloudflare account under DNS Records

> ⚠️ **Execution Note**
>
>The following command requires:
>- A registered domain
>- An authenticated Cloudflare session
>
>If you are reading this as a demo, **do not run this cell**.

In [ ]:
# Delete old certificates
!rm /root/.cloudflared/cert.pem

In [ ]:
# Authorize Cloudflare tunnel for domain
!./cloudflared tunnel login

In [ ]:
#verify certificate
!ls -l /root/.cloudflared

In [ ]:
# Delete old tunnels
!./cloudflared tunnel delete <TUNNEL_NAME>

>🌐 **Step 5: Creating a Secure Tunnel**
>
>Create a safe Cloudflare Tunnel for your website.


>⚠️ **Execution Note**
>
>The following command requires:
>- A registered domain
>- An authenticated Cloudflare session
>
>If you are reading this as a demo, **do not run this cell**.

In [ ]:
#Create the tunnel
# Replace <TUNNEL_NAME> with your own tunnel name
./cloudflared tunnel create <TUNNEL_NAME>


>💡 **Tip:** Save the tunnel credentials `.json` file for future use.

In [ ]:
# check the json file name
!ls /root/.cloudflared


In [ ]:
# Prepare config file
# Replace <TUNNEL_UUID> with your own tunnel UUID and <DOMAIN_NAME> with your own domain name
%%bash
cat << EOF > /root/.cloudflared/config.yml
tunnel: <TUNNEL_UUID>
credentials-file: /root/.cloudflared/<TUNNEL_UUID>.json

ingress:
  - hostname: tunnel.<DOMAIN_NAME>
    service: http://localhost:5000
  - service: http_status:404
EOF

In [ ]:
#check if file was created sucessfully
!cat /root/.cloudflared/config.yml

In [ ]:
# Restart the httpbin
!pkill gunicorn
!nohup gunicorn httpbin:app -b 127.0.0.1:5000 &

In [ ]:
# Rerun the tunnel in the background
# Replace <TUNNEL_NAME> with your own tunnel name
!pkill cloudflared
!nohup ./cloudflared tunnel run <TUNNEL_NAME> &

>**Check the logs to see if everything is working.**

In [ ]:
# Check the logs
!tail -n 30 nohup.out

>### Expected Output
>
>If the tunnel starts successfully, you should see logs indicating registered tunnel connections

In [ ]:
# Check all tunnels and connections
!./cloudflared tunnel list

>🔗 **Step 6: Linking the Domain**
>
>Connect the tunnel to a domain so that visitors can reach your website using a public web address.
>
>Direct your DNS record in Cloudflare to this json file

>⚠️ **Execution Note**
>
>The following command requires:
>- A registered domain
>- An authenticated Cloudflare session
>
>If you are reading this as a demo, **do not run this cell**.

In [ ]:
# Create a DNS record in Cloudflare

# Replace <TUNNEL_NAME> with your own tunnel name and <DOMAIN_NAME> with your domain name
!./cloudflared tunnel route dns <TUNNEL_NAME> tunnel.<DOMAIN_NAME>

>🧪 **Step 7: Checking Everything Works**
>
>Check if the website is accessible through your public URL and that it returns all HTTP headers.

In [ ]:
# Replace <DOMAIN_NAME> with your own registered domain.
tunnel.<DOMAIN_NAME>/headers

>### Expected Response
>
>A successful request returns a JSON payload containing HTTP headers, such as:
>
>- `Host`
>- `CF-Connecting-IP`
>- `User-Agent`

>## 🎯 Intended Audience
>
>This notebook is intended for Developers new to Cloudflare Tunnel

---

✍️ **Created by:** Katrin Mrotzeck  
📦 **Notebook Version:** v1.0.0  
🛠️ **Environment:** Google Colab (Python 3)